In [23]:
import pandas as pd

# read the dataset
data = pd.read_excel(r"C:\Users\zhang\Desktop\interexp.xlsx")

# get a glance on data info
print(data.info())
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A       41 non-null     float64
 1   B       43 non-null     float64
dtypes: float64(2)
memory usage: 1.0 KB
None
       A      B
0  25.33  26.45
1  26.77  27.53
2  22.76  20.02
3  20.94  22.83
4  25.40  28.05


In [24]:
print(data.isnull().sum())  # count NA value


A    17
B    15
dtype: int64


In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

# Step 3: Simulating the EM algorithm for missing value imputation
def em_algorithm(data_missing, max_iter=100):
    n_samples, n_features = data_missing.shape
    
    # Step 4: Initialize the mean and covariance of the complete data
    # Use the means and covariances of the available data for initialization
    data_complete = data_missing.copy()
    data_complete[np.isnan(data_complete)] = np.nanmean(data_missing, axis=0)
    
    mean_estimate = np.nanmean(data_complete, axis=0)
    cov_estimate = np.cov(data_complete.T, nan_policy='omit')
    
    for iteration in range(max_iter):
        # Step 5: E-step (Impute missing values using current estimates)
        for i in range(n_samples):
            if np.any(np.isnan(data_missing[i])):
                # Predict missing values based on current mean and covariance
                missing_indices = np.where(np.isnan(data_missing[i]))[0]
                observed_indices = np.where(~np.isnan(data_missing[i]))[0]
                
                # Conditioning on observed values
                mu_cond = mean_estimate[observed_indices]
                cov_cond = cov_estimate[np.ix_(observed_indices, observed_indices)]
                cov_cross = cov_estimate[np.ix_(observed_indices, missing_indices)]
                
                # Impute missing values using conditional expectation
                inv_cov_cond = np.linalg.inv(cov_cond)
                mean_missing_given_observed = mu_cond + cov_cross.dot(inv_cov_cond).dot(data_missing[i, observed_indices] - mu_cond)
                
                data_missing[i, missing_indices] = mean_missing_given_observed
                
        # Step 6: M-step (Update estimates of the mean and covariance)
        mean_estimate = np.nanmean(data_missing, axis=0)
        cov_estimate = np.cov(data_missing.T, nan_policy='omit')
    
    return data_missing

# Step 7: Prepare the data for EM imputation
# Assume that 'Stimulus_A' and 'Stimulus_B' are the column names in the interexp dataset.
# If your dataset uses different column names, adjust the code accordingly.
data_missing = data[['Stimulus_A', 'Stimulus_B']].values

# Step 8: Run the EM algorithm to impute missing values
data_imputed = em_algorithm(data_missing)

# Step 9: Visualize the distributions before and after imputation

# Before imputation (with missing values)
plt.figure(figsize=(12, 6))

# Before imputation
plt.subplot(1, 2, 1)
plt.scatter(data_missing[:, 0], data_missing[:, 1], alpha=0.5, label='Missing Data')
plt.title('Before Imputation (Missing Values)')
plt.xlabel('Stimulus A')
plt.ylabel('Stimulus B')
plt.grid(True)

# After imputation
plt.subplot(1, 2, 2)
plt.scatter(data_imputed[:, 0], data_imputed[:, 1], alpha=0.5, label='Imputed Data', color='orange')
plt.title('After Imputation (Imputed Values)')
plt.xlabel('Stimulus A')
plt.ylabel('Stimulus B')
plt.grid(True)

plt.tight_layout()
plt.show()

# Step 10: Ensure reproducibility of results
np.random.seed(42)





KeyError: "None of [Index(['Stimulus_A', 'Stimulus_B'], dtype='object')] are in the [columns]"